# simple GenAI app

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]= os.getenv("LANGCHAIN_PROJECT")

In [ ]:
# data ingestion
from langchain_community.document_loaders import WebBaseLoader
loader= WebBaseLoader(web_path="https://python.langchain.com/docs/versions/v0_3/")
doc=loader.load()

In [5]:
len(doc)

1

In [6]:
# text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
split_doc=text_splitter.split_documents(doc)

In [8]:
len(split_doc)

44

In [9]:
# embedding and vector store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings=OpenAIEmbeddings(
    model="text-embedding-3-small"
)

db=FAISS.from_documents(split_doc,embeddings)
db

In [10]:
# query from a vector store db
query="Whats changes in v0.3"
result=db.similarity_search(query)
result

[Document(id='b36e6ac8-33cf-4bba-959f-24953e00b3e4', metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content="Last updated: 09.16.24\nWhat's changed\u200b"),
 Document(id='560b8542-d901-4120-8c39-618af3611a34', metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='Added utilities for interacting with chat models: universal model constructor, rate limiter, message utilities,\nAdded the ability to dispatch custom events.\nRevamped integration docs and API reference. Read more here.\nMarked as deprecated a number of legacy chains and added migration guides for all of them. These are slated for removal in langchain 1.0.0. See the deprecated chains and associated migration guides here.'),
 Document(id='6f97b60e-20dc-476d-b7b9-2e

In [11]:
result[0].page_content

"Last updated: 09.16.24\nWhat's changed\u200b"

In [12]:
result[1].page_content

'Added utilities for interacting with chat models: universal model constructor, rate limiter, message utilities,\nAdded the ability to dispatch custom events.\nRevamped integration docs and API reference. Read more here.\nMarked as deprecated a number of legacy chains and added migration guides for all of them. These are slated for removal in langchain 1.0.0. See the deprecated chains and associated migration guides here.'

In [ ]:
## Retrieval Chain, Document Chain 
